## apt-get install ffmpeg

In [8]:
import os
import glob
import cv2
import shutil
from multiprocessing import Pool
from functools import partial
from tqdm import tqdm
import numpy as np

In [9]:
def auto_unzip_fun(x, f):
    return f(*x)



def make_video(output_mp4_path, img_path_list, save_frames_dir=None, fps=24):

    """
    output_path is the final mp4 name
    img_dir is where the images to make into video are saved.
    """

    first_img = cv2.imread(img_path_list[0])
    h, w = first_img.shape[:2]

    pool_size = 40
    tmp_avi_video_path = '%s.avi' % output_mp4_path
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    videoWriter = cv2.VideoWriter(tmp_avi_video_path, fourcc, fps, (w, h))
    args_list = [(img_path,) for img_path in img_path_list]
    with Pool(pool_size) as p:
        for img in tqdm(p.imap(partial(auto_unzip_fun, f=cv2.imread), args_list), total=len(args_list)):
            videoWriter.write(img)
    videoWriter.release()

    if save_frames_dir:
        for i, img_path in enumerate(img_path_list):
            shutil.copy(img_path, '%s/%.8d.jpg' % (save_frames_dir, i))

    os.system("ffmpeg -y -i %s -vcodec h264 %s > /dev/null 2>&1" % (tmp_avi_video_path, output_mp4_path))
    os.system("rm %s" % (tmp_avi_video_path))

### save_dir ： 填入要生成影片的位置
### output_path ：主要加上mp4
### src_dir ： 填入要转换的图片文件夹


In [10]:
save_dir = '/Users/stephenfang/Desktop/workplace/'
output_path = os.path.join(save_dir, 'test.mp4')

test_dir = '/Users/stephenfang/Desktop/impersonator/samples/refs/iPER/024_8_2'
img_path_list = sorted(glob.glob('%s/*.jpg' % src_dir))
make_video(output_path, img_path_list)

100%|██████████| 1508/1508 [00:17<00:00, 85.17it/s] 
